In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# fetch_player_stats

> Fetch player starts for individual games.

In [ ]:
#| default_exp fetch_player_stats

In [ ]:
#| hide
from nbdev.showdoc import *
import pandas as pd
from typing import Optional
import os
import requests

In [ ]:
#| export
def fetch_player_stats(player_id: str, date: Optional[str] = None):
    """
    Fetches all player stats for a particular player or if a date is passed then just for a particular game.
    """
    if date:
        print(f"Fetching player stats for player_id:{player_id} on date:{date}...")
    else:
        print(f"Fetching all player stats for player_id:{player_id}...")
    try:
        if date:
            player_url = f"https://github.com/seanyboi/rugbydata/blob/main/data/player/{player_id}/{date}"
            path = requests.get(player_url)
            urls = [f"https://github.com/seanyboi/rugbydata/blob/main/{p['path']}?raw=true" for p in path.json()["payload"]["tree"]["items"]]
            player_stats = pd.concat((pd.read_parquet(u, engine="pyarrow") for u in urls))
            return player_stats
        else:
            player_url = f"https://github.com/seanyboi/rugbydata/blob/main/data/player/{player_id}"
            path = requests.get(player_url)
            date_urls = [f"https://github.com/seanyboi/rugbydata/blob/main/{p['path']}" for p in path.json()["payload"]["tree"]["items"]]
            player_urls = [requests.get(url) for url in date_urls]
            player_urls = [p.json()["payload"]["tree"]["items"][0]["path"] for p in player_urls]
            player_stats_url = [f"https://github.com/seanyboi/rugbydata/blob/main/{p}?raw=true" for p in player_urls]
            player_stats = pd.concat((pd.read_parquet(u, engine="pyarrow") for u in player_stats_url))
            return player_stats
    except Exception as e:
        print(f"No player stats for {player_id} because the player id does not exist. Please raise an issue! - {e}")

In [ ]:
fetch_player_stats(player_id="102049")

Fetching all player stats for player_id:102049...


In [ ]:
fetch_player_stats(player_id="102049", date="20230128")

Fetching player stats for 102049 on 20230128...


,player_id,game_date,name,team,team_id,competition_id,competition,team_vs,team_vs_id,weight,...,rucks_won,runs,tackles,total_free_kicks_conceded,total_lineouts,tries,try_assists,turnover_knock_on,turnovers_conceded,yellow_cards
0,102049,20230128,Billy Twelvetrees,Gloucester Rugby,25900,267979,Premiership Rugby,116227,Exeter Chiefs,101.0,...,3.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()